### Whale & Dolphin Cropped Images
* Version: 1.0
* Creator: Eli Kuo
* Environment: python 3.8.8

Utility scripts for visualization of "Happywhale - Whale and Dolphin Identification" competition dataset, which is defined in the other [kernel](https://www.kaggle.com/code/acchiko/utility-functions-for-visualization-of-dataset). 


In [2]:
# Libs import
import utility_functions_for_visualization_of_dataset as myutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from tqdm import tqdm

In [3]:
# Defines class for processing train/test images and metadata.
class WhaleAndDolphin():
    def __init__(self, path_to_metadata, path_to_dir_images, \
                 path_to_dir_nobg_images, path_to_dir_cropped_images):
        self._path_to_metadata = path_to_metadata
        self._path_to_dir_images = path_to_dir_images
        self._path_to_dir_nobg_images = path_to_dir_nobg_images
        self._path_to_dir_cropped_images = path_to_dir_cropped_images
        
        # Loads metadata to variable "_metadata_all"
        self._metadata_all = pd.read_csv(path_to_metadata)
        
        # Adds several colmuns.
        path_to_images = \
            ["%s/%s" % (path_to_dir_images, row.image) \
             for row in self._metadata_all.itertuples()]
        self._metadata_all["path_to_image"] = path_to_images
        
        path_to_nobg_images = \
            ["%s/%s" % (path_to_dir_nobg_images, row.image.replace(".jpg", ".png")) \
             for row in self._metadata_all.itertuples()]
        self._metadata_all["path_to_nobg_image"] = path_to_nobg_images
        
        path_to_cropped_images = \
            ["%s/%s" % (path_to_dir_cropped_images, row.image.replace(".jpg", ".png")) \
             for row in self._metadata_all.itertuples()]
        self._metadata_all["path_to_cropped_image"] = path_to_cropped_images
        
        annotations_xyxy = \
            [[] for row in self._metadata_all.itertuples()]
        self._metadata_all["annotations_xyxy"] = annotations_xyxy
        
        # Copies the metadata for processing it.
        self._metadata = self._metadata_all.copy()
        
        self._all_species = self.getSpecies()
        self._all_individual_ids = self.getIndividualIDs()
        
    def resetMetadata(self, initialize=False):
        if hasattr(self, "_metadata_tmp") and not initialize:
            self._metadata = self._metadata_tmp.copy()
        else:
            self._metadata = self._metadata_all.copy()
            
    def saveMetadataTemporary(self):
        self._metadata_tmp = self._metadata.copy()
        
    def filterMetadata(self, query="index > -1"):
        sliced_metadata = \
            self._metadata.query(query).reset_index(drop=True)
        self._metadata = sliced_metadata.copy()
        
    def filterMetadataBackgroundRemovedImageExistence(self):
        indices = []
        for row in self._metadata.itertuples():
            if not os.path.exists(row.path_to_nobg_image):
                indices.append(row.Index)
                
        sliced_metadata = self._metadata.drop(index=indices).reset_index(drop=True)
        self._metadata = sliced_metadata.copy()
        
    def writeMetadata(self, path_to_metadata):
        self._metadata.to_csv(path_to_metadata, index=False)
        
    def getMetadata(self):
        return self._metadata
    
    def getSpecies(self):
        return self._metadata["species"].unique()
    
    def _species2id(self, species):
        return np.where(self._all_species == species)[0][0]
    
    def getIndividualIDs(self):
        return self._metadata["individual_id"].unique()
    
    def showImagesTile(self, num_cols=4, draw_annotations=False):
        metadata = self._metadata
        titles = [row.image for row in metadata.itertuples()]
        path_to_images = [row.path_to_image \
                          for row in metadata.itertuples()]
        images = myutils.getImages(path_to_images)
        if "annotations_xyxy" in metadata.columns and draw_annotations:
            annotations_xyxy_for_images = [row.annotations_xyxy \
                                           for row in metadata.itertuples()]
            texts_for_images = [["" for _ in \
                                 range(len(row.annotations_xyxy))] \
                                 for row in metadata.itertuples()]
            myutils.drawAnnotations( \
                images, \
                annotations_xyxy_for_images=annotations_xyxy_for_images, \
                texts_for_images=texts_for_images, \
                line_color="green", line_width=3, text_color="green" \
            )
        myutils.showImagesTile(titles, images, num_cols=num_cols)
        
    def showProcessedImagesTile(self, num_cols=3, draw_annotations=False):
        metadata = self._metadata
        titles, path_to_images = [], []
        for row in metadata.itertuples():
            titles.append("%s (Org.)" % row.image)
            path_to_images.append(row.path_to_image)
            
            titles.append("%s (BG. removed)" % row.image)
            path_to_images.append(row.path_to_nobg_image)
            
            titles.append("%s (Cropped)" % row.image)
            path_to_images.append(row.path_to_cropped_image)
        
        images = myutils.getImages(path_to_images)
        if "annotations_xyxy" in metadata.columns and draw_annotations:
            annotations_xyxy_for_images = [row.annotations_xyxy \
                                           for row in metadata.itertuples()]
            texts_for_images = [["" for _ in \
                                 range(len(row.annotations_xyxy))] for row in metadata.itertuples()]
            myutils.drawAnnotations(
                images[::3],
                annotations_xyxy_for_images=annotations_xyxy_for_images,
                texts_for_images=texts_for_images, line_color="red", line_width=3, text_color="red" 
            ) # For only org. images.
        myutils.showImagesTile(titles, images, num_cols=num_cols)
        
    def showIndividualImagesTile(self, num_cols=4, max_num_individual_images=4, max_num_individuals=10, draw_annotations=False):
        self.saveMetadataTemporary()
        
        individual_ids = self.getIndividualIDs()
        for individual_id in individual_ids[:max_num_individuals]:
            print()
            print("Individual ID : %s" % individual_id)
            self.filterMetadata(query="individual_id == \"%s\"" % individual_id)
            self.filterMetadata(query="index < %d" % max_num_individual_images)
            self.showImagesTile(num_cols=num_cols, draw_annotations=draw_annotations
            )
            self.resetMetadata()
            
    def removeBackground(self):
        metadata = self._metadata
        path_to_inputs = [row.path_to_image for row in metadata.itertuples()]
        path_to_outputs = [row.path_to_nobg_image for row in metadata.itertuples()]
        
        for path_to_input, path_to_output in zip(path_to_inputs, path_to_outputs):
            # Input to Output
            !backgroundremover -i {path_to_input} -o {path_to_output}
        
    def calculateAnnotationsXyxy(self):
        batch_size = 100
        num_batches = len(self._metadata) // batch_size + 1
        
        for i_batch in range(num_batches):
            i_start = i_batch * batch_size
            i_end = i_start + batch_size
            metadata = self._metadata.iloc[i_start:i_end]
            
            path_to_nobg_images = [row.path_to_nobg_image for row in metadata.itertuples()]
            nobg_images = myutils.getImages(path_to_nobg_images)
            ## fixed_images = [Image.eval(nobg_image, self._removeBugPixel) for nobg_image in nobg_images]
            class_ids = [self._species2id(row.species) for row in metadata.itertuples()]
            
            annotations_xyxy = []
            for nobg_image, class_id in zip(nobg_images, class_ids):
                _, _, _, a = nobg_image.split()
                x_min, y_min, x_max, y_max = a.getbbox() # Bounding box of non-zero alpha region
                # confidence is for the further accuraccy attribute to annalysis 
                confidence = 1.0
                annotation_xyxy = myutils._annotationXyxy(class_id, x_min, y_min, x_max, y_max, confidence)
                annotations_xyxy.append([annotation_xyxy])
                
            self._metadata["annotations_xyxy"].iloc[i_start:i_end] = annotations_xyxy
    
    #def _removeBugPixel(self, pixel_value):
    #    if pixel_value == 1:
    #        return 0
    #    else:
    #        return pixel_value
        
    def cropObject(self):
        batch_size = 100
        num_batches = len(self._metadata) // batch_size + 1
        
        for i_batch in range(num_batches):
            i_start = i_batch * batch_size
            i_end = i_start + batch_size
            metadata = self._metadata.iloc[i_start:i_end]
            
            path_to_inputs = [row.path_to_image for row in metadata.itertuples()]
            path_to_outputs = [row.path_to_cropped_image for row in metadata.itertuples()]
            annotations_xyxy = [row.annotations_xyxy for row in metadata.itertuples()]
            
            for path_to_input, path_to_output, annotations_xyxy in zip(path_to_inputs, path_to_outputs, annotations_xyxy):
                
                image = Image.open(path_to_input)
                annotation_xyxy = \
                    self._maxConfidenceAnnotation(annotations_xyxy)
                x_min = annotation_xyxy["x_min"]
                y_min = annotation_xyxy["y_min"]
                x_max = annotation_xyxy["x_max"]
                y_max = annotation_xyxy["y_max"]
                image_cropped = image.crop((x_min, y_min, x_max, y_max))
                image_cropped.save(path_to_output)
            
    def _maxConfidenceAnnotation(self, annotations_xyxy):
        confidences = np.array([annotation_xyxy["confidence"] for annotation_xyxy in annotations_xyxy])
        index = np.argmax(confidences)
        return annotations_xyxy[index]

### Outprint for training data

In [52]:
# Loads metadata for train images.
whale_and_dolphin = WhaleAndDolphin(
    path_to_metadata = "Data/train.csv",
    path_to_dir_images = "Data/train_images",
    # No background
    path_to_dir_nobg_images = "Data/no_background/train_images",
    # Cropped images
    path_to_dir_cropped_images = "Data/cropped/train_images"
)

In [ ]:
# Shows train images for the first 3 individuals for each species.
num_cols = 4
max_num_individual_images = 4
max_num_individuals = 3

all_species = whale_and_dolphin.getSpecies()
for i, species in enumerate(all_species):
    whale_and_dolphin.filterMetadata(query="species == \"%s\"" % species)
    
    print("\n--------------------------------------------------\n")
    print("\n   Images for species No.%02d %s \n" % (i, species))
    print("\n--------------------------------------------------\n")
    whale_and_dolphin.showIndividualImagesTile(
        num_cols=num_cols,
        max_num_individual_images=max_num_individual_images,
        max_num_individuals=max_num_individuals
    )
    
    whale_and_dolphin.resetMetadata(initialize=True)

### Step 1: Romving Background

In [53]:
# Removes background. Limits number of processing images because of kaggle notebook limitation. Sets same number for each species as possible.
num_images_per_species = int(4000 / len(all_species))
i_start = 0
i_end = i_start + num_images_per_species
all_species = whale_and_dolphin.getSpecies()
whale_and_dolphin.saveMetadataTemporary()

for i, species in enumerate(all_species):
   whale_and_dolphin.filterMetadata(query="species == \"%s\"" % species)
   whale_and_dolphin.filterMetadata(query="%d <= index < %d" % (i_start, i_end))
   print("\n--------------------------------------------------\n")
   print("\n   Processing images for species No.%02d %s\n" % (i, species))
   print("\n--------------------------------------------------\n")
   # whale_and_dolphin.removeBackground()
   whale_and_dolphin.resetMetadata()


--------------------------------------------------


   Processing images for species No.00 melon_headed_whale


--------------------------------------------------


--------------------------------------------------


   Processing images for species No.01 humpback_whale


--------------------------------------------------


--------------------------------------------------


   Processing images for species No.02 false_killer_whale


--------------------------------------------------


--------------------------------------------------


   Processing images for species No.03 bottlenose_dolphin


--------------------------------------------------


--------------------------------------------------


   Processing images for species No.04 beluga


--------------------------------------------------


--------------------------------------------------


   Processing images for species No.05 minke_whale


--------------------------------------------------


--------------------------

### Step 2: Adding Bounding Box
Bounding box is cropped for future work (The model for identifing whale and dolphin may be created using the cropped images.) By using the background removing we are able to access the content of cropping content.

In [ ]:
# Calculates bounding box of object.
whale_and_dolphin.calculateAnnotationsXyxy()

In [55]:
# Filters metadata, which has background removed image.
whale_and_dolphin.filterMetadataBackgroundRemovedImageExistence()
metadata = whale_and_dolphin.getMetadata()
# Then saves metadata as csv file.
path_to_train_metadata = "Data/train_with_annotation.csv"
whale_and_dolphin.writeMetadata(path_to_train_metadata)
metadata.head()

,image,species,individual_id,path_to_image,path_to_nobg_image,path_to_cropped_image,annotations_xyxy
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,Data/train_images/00021adfb725ed.jpg,Data/no_background/train_images/00021adfb725ed...,Data/cropped/train_images/00021adfb725ed.png,"[{'class_id': 0, 'x_min': 0, 'y_min': 111, 'x_..."
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,Data/train_images/000562241d384d.jpg,Data/no_background/train_images/000562241d384d...,Data/cropped/train_images/000562241d384d.png,"[{'class_id': 1, 'x_min': 0, 'y_min': 139, 'x_..."
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,Data/train_images/0007c33415ce37.jpg,Data/no_background/train_images/0007c33415ce37...,Data/cropped/train_images/0007c33415ce37.png,"[{'class_id': 2, 'x_min': 0, 'y_min': 0, 'x_ma..."
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,Data/train_images/0007d9bca26a99.jpg,Data/no_background/train_images/0007d9bca26a99...,Data/cropped/train_images/0007d9bca26a99.png,"[{'class_id': 3, 'x_min': 0, 'y_min': 79, 'x_m..."
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,Data/train_images/00087baf5cef7a.jpg,Data/no_background/train_images/00087baf5cef7a...,Data/cropped/train_images/00087baf5cef7a.png,"[{'class_id': 1, 'x_min': 0, 'y_min': 1349, 'x..."


In [57]:
# Get the total number of the files in the directory
path, dirs, files = next(os.walk('Data/no_background/train_images'))
num_images_train = len(files)
i_start = 2 * num_images_train
i_end = i_start + num_images_train
# Locating the position
whale_and_dolphin.filterMetadata(query="%d <= index < %d" % (i_start, i_end))
whale_and_dolphin.cropObject()

In [58]:
# Shows first 100 processed images.
num_images_per_batch = 100
metadata = whale_and_dolphin.getMetadata()
num_batches = int(len(metadata) / num_images_per_batch)
max_num_batches = min(num_batches, 1)
num_cols = 3

whale_and_dolphin.saveMetadataTemporary()
for i in range(max_num_batches):
    i_start = i * num_images_per_batch
    i_end = i_start + num_images_per_batch
    whale_and_dolphin.showProcessedImagesTile(num_cols=num_cols, draw_annotations=True)
    whale_and_dolphin.resetMetadata()